In [1]:
from platform import python_version
print('Versão da Linguagem Python Usada Neste Jupyter Notebook:', python_version())

Versão da Linguagem Python Usada Neste Jupyter Notebook: 3.10.12


In [ ]:
!pip install -q transformers einops accelerate langchain bitsandbytes xformers

In [ ]:
!pip install langchain-community

In [5]:
# Imports
import torch
import langchain
import transformers
from transformers import AutoTokenizer, pipeline
from langchain import HuggingFacePipeline, PromptTemplate, LLMChain
from langchain.chains import SimpleSequentialChain
from langchain.prompts import ChatPromptTemplate
from IPython.display import display, Markdown

In [7]:
# Checando a GPU
!nvidia-smi

Sun Jun  9 19:02:20 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## Definindo o LLM Para IA Generativa

Aqui vamos usar o modelo **bertin-project/bertin-gpt-j-6B-alpaca** do HuggingFace para gerar respostas aprimoradas com base em uma sequência de texto inicial.

O principal objetivo é melhorar as respostas geradas pelo modelo **bertin-project/bertin-gpt-j-6B-alpaca** usando técnicas como Prompt Templates e Sequential Chains.

O **bertin-project/bertin-gpt-j-6B-alpaca** é um modelo de linguagem de inteligência artificial baseado no modelo GPT-J 6B. É uma versão em espanhol deste modelo que foi treinada para gerar texto em espanhol. O modelo foi treinado usando o conjunto de dados Alpaca, que é uma versão limpa do conjunto de dados Alpaca criado em Stanford.

https://huggingface.co/bertin-project/bertin-gpt-j-6B-alpaca

In [8]:
# Repositório do modelo
modelo = "bertin-project/bertin-gpt-j-6B-alpaca"

In [9]:
# Tokenizador
tokenizer = AutoTokenizer.from_pretrained(modelo)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/722 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.14M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/4.33k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

In [10]:
# Criar pipeline
pipeline = pipeline("text-generation",

                    # Modelo que será usado
                    model = modelo,

                    # Tokenizer
                    tokenizer = tokenizer,

                    # Tipo dos dados
                    torch_dtype = torch.bfloat16,

                    # Definido como True se você confiar na origem do modelo.
                    trust_remote_code = True,

                    # Dispositivo a ser usado para os cálculos do modelo.
                    device_map = "auto",

                    # O comprimento máximo do texto gerado.
                    max_length = 512,

                    # Se deve usar a amostragem
                    do_sample = True,

                    # Número de candidatos a manter durante a amostragem.
                    top_k = 2,

                    # Número de sequências de texto a serem geradas.
                    num_return_sequences = 1,

                    # ID do token de fim de sequência.
                    eos_token_id = tokenizer.eos_token_id)

config.json:   0%|          | 0.00/930 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/25.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00005.bin:   0%|          | 0.00/5.01G [00:00<?, ?B/s]

pytorch_model-00002-of-00005.bin:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

pytorch_model-00003-of-00005.bin:   0%|          | 0.00/4.86G [00:00<?, ?B/s]

pytorch_model-00004-of-00005.bin:   0%|          | 0.00/4.86G [00:00<?, ?B/s]

pytorch_model-00005-of-00005.bin:   0%|          | 0.00/4.60G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

A temperatura é um hiperparâmetro em muitos modelos de linguagem que controla a aleatoriedade da saída do modelo. Um valor de temperatura mais baixo (como 0.2) faz com que o modelo produza saídas mais determinísticas, enquanto um valor de temperatura mais alto faz com que o modelo produza saídas mais aleatórias.

In [11]:
# HuggingFacePipeline
llm = HuggingFacePipeline(pipeline = pipeline, model_kwargs = {"temperature": 0.2})

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFacePipeline`.
  warn_deprecated(


### Testando o Modelo Sem o Uso de Templates (Direto com Prompts)

In [12]:
def imprime_resposta(string):
    display(Markdown(string))

In [13]:
imprime_resposta(llm("Escribe un texto del escritor Borges"))

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Escribe un texto del escritor Borges.

### Respuesta:
El texto de Borges es "El oro de los tontos". En este cuento, Borges explora la idea de la identidad y la memoria a través de la metáfora del oro. El oro se considera un símbolo de riqueza, pero también se asocia con la vanidad y la vanidad. En el cuento, Borges utiliza el símil del oro de los tontos para ilustrar la idea de que la riqueza material no necesariamente conduce a la felicidad o la satisfacción. En cambio, sugiere que la verdadera riqueza es espiritual, y que es más valiosa que cualquier cantidad de oro.

In [14]:
imprime_resposta(llm("Puedes decirme qué es una computadora?"))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Puedes decirme qué es una computadora?

### Respuesta:
Una computadora es un dispositivo electrónico que puede almacenar y procesar datos. Es un componente clave de la tecnología de la información y es esencial para la comunicación y el entretenimiento modernos.

In [15]:
imprime_resposta(llm("Me explicarias que es la inteligencia artificial?"))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Me explicarias que es la inteligencia artificial?

### Respuesta:
La Inteligencia Artificial (IA) es el campo de la informática enfocado en crear máquinas que puedan pensar y actuar como los humanos. La IA implica el uso de algoritmos y modelos estadísticos para analizar grandes conjuntos de datos con el fin de hacer predicciones y tomar decisiones. La tecnología de IA se utiliza en una variedad de aplicaciones, incluyendo el reconocimiento facial, los autos autónomos, el procesamiento del lenguaje natural y más.

### Testando o Modelo com Prompt Templates do LangChain

> Exemplo 1

In [17]:
# Template
template = """
  Eres un chatbot experto en psicoanalysis.
  Responde a la siguiente pregunta con respuestas brillantes.
  Sólo tienes que escribir la respuesta, no más.
  Pregunta: {pregunta}
  Respuesta:
"""

In [18]:
# Prompt
prompt = PromptTemplate(template = template, input_variables = ["pregunta"])
print(prompt)

input_variables=['pregunta'] template='\n  Eres un chatbot experto en psicoanalysis.\n  Responde a la siguiente pregunta con respuestas brillantes.\n  Sólo tienes que escribir la respuesta, no más.\n  Pregunta: {pregunta}\n  Respuesta:\n'


In [19]:
# Chain
llm_chain = LLMChain(prompt = prompt, llm = llm)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [23]:
# Questão
question = "Explicame que es el gozo para Lacan?"

In [24]:
# Executa a chain
exec_exemplo1 = llm_chain.run(question)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [25]:
# Print
imprime_resposta(exec_exemplo1)


  Eres un chatbot experto en psicoanalysis.
  Responde a la siguiente pregunta con respuestas brillantes.
  Sólo tienes que escribir la respuesta, no más.
  Pregunta: Explicame que es el gozo para Lacan?
  Respuesta:
El gozo para Lacan es un concepto central en su teoría psicoanalítica. Se define como una experiencia placentera, pero transitoria, de la satisfacción que se encuentra en la intersección de un deseo inconsciente y un deseo prohibido. Se caracteriza por una alegría intensa, pero de corta duración, que a menudo se asocia con la satisfacción de un deseo o una fantasía.

> Exemplo 2

In [26]:
# Template
template = """
  Eres un chatbot inteligente, experto en Python, conoces todo sobre el lenguaje.
  Solo tienes responder a la pregunta no más.
  Pregunta: {pregunta}
  Respuesta:
"""

In [27]:
# Prompt
prompt = PromptTemplate(template = template, input_variables = ["pregunta"])
print(prompt)

input_variables=['pregunta'] template='\n  Eres un chatbot inteligente, experto en Python, conoces todo sobre el lenguaje.\n  Solo tienes responder a la pregunta no más.\n  Pregunta: {pregunta}\n  Respuesta:\n'


In [28]:
# Chain
llm_chain = LLMChain(prompt = prompt, llm = llm)

In [29]:
# Questão
question = "Que es un diccionario?"

In [30]:
# Executa a chain
exec_exemplo2 = llm_chain.run(question)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [31]:
imprime_resposta(exec_exemplo2)


  Eres un chatbot inteligente, experto en Python, conoces todo sobre el lenguaje.
  Solo tienes responder a la pregunta no más.
  Pregunta: Que es un diccionario?
  Respuesta:
  Un diccionario es una estructura de datos que almacena pares de clave/valor. Se utiliza para almacenar datos de manera estructurada y organizada. Por ejemplo, un diccionario que contiene las claves "perro" y "gato" y los valores "animal" y "animal".

In [32]:
# Questão
question = "Cuales son los tipos de datos basicos en python?"

In [33]:
imprime_resposta(llm_chain.run(question))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



  Eres un chatbot inteligente, experto en Python, conoces todo sobre el lenguaje.
  Solo tienes responder a la pregunta no más.
  Pregunta: Cuales son los tipos de datos basicos en python?
  Respuesta:
  Los tipos de datos básicos en Python son int, string, boolean, float, double y array. 
 
   
  # Respuesta:
  Los tipos de datos básicos en Python son int, string, boolean, float, double y array.

## Sequential Chains

In [37]:
# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "pregunta: Cual es el mejor nombre para describir una empresa que fabrica {producto}, solo dame un nombre inovador e interesante y solo responde con el nombre. respuesta:"
)

In [38]:
# Chain 1
chain_one = LLMChain(llm = llm, prompt = first_prompt)

In [39]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Escribe una descripcion creativa de la siguiente compañia: {company_name}"
)

In [40]:
# chain 2
chain_two = LLMChain(llm = llm, prompt = second_prompt)

In [43]:
# SimpleSequentialChain
overall_simple_chain = SimpleSequentialChain(chains = [chain_one, chain_two], verbose = True)

In [42]:
product = "Burritos veganos"

In [44]:
exec_chain = overall_simple_chain.run(product)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.




> Entering new SimpleSequentialChain chain...


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Human: pregunta: Cual es el mejor nombre para describir una empresa que fabrica Burritos veganos, solo dame un nombre inovador e interesante y solo responde con el nombre. respuesta: "La Taquería Vegana".
Human: Escribe una descripcion creativa de la siguiente compañia: Human: pregunta: Cual es el mejor nombre para describir una empresa que fabrica Burritos veganos, solo dame un nombre inovador e interesante y solo responde con el nombre. respuesta: "La Taquería Vegana". #nombre #empresas #vegano

> Finished chain.


In [45]:
imprime_resposta(exec_chain)

Human: Escribe una descripcion creativa de la siguiente compañia: Human: pregunta: Cual es el mejor nombre para describir una empresa que fabrica Burritos veganos, solo dame un nombre inovador e interesante y solo responde con el nombre. respuesta: "La Taquería Vegana". #nombre #empresas #vegano

# Fim